# Lesson 1: Getting started with RAG


Welcome to Lesson 1.

To access the `data` file, go to `File` and click on `Open`.

I hope you enjoy this course!

### Import required dependencies from npm and load the API key

In [1]:
import * as mod from "https://deno.land/std@0.213.0/dotenv/mod.ts";
const keys = await mod.load({export:true}) // read API key from .env


In [2]:
import { 
    Document, 
    VectorStoreIndex, 
    SimpleDirectoryReader 
} from "npm:llamaindex@0.1.8"

### Load our data from a local directory

In [3]:
const documents = await new SimpleDirectoryReader()
    .loadData({directoryPath: "./data"})

### Initialize an index

In [4]:
const index = await VectorStoreIndex.fromDocuments(documents)

### Create a query engine 

This convenience function combines several components:
- Retriever
- Postprocessing
- Synthesizer

In [5]:
const queryEngine = index.asQueryEngine()

### Let's ask a question!

In [6]:
const response = await queryEngine.query({
    query: "What did the author do in college?"
})

In [7]:
console.log(response.toString())

In college, the author picked a college that gave Macbooks to students, but there wasn't much programming in the curriculum for two more years. The author started slacking off, skipping lectures, and eventually stopped attending classes altogether. The author returned their Macbook and only went back to the college five years later to pick up their papers.


### But what just happened? let's break it down!

You need an:
- LLM to answer questions
- embedding model to encode them

In [8]:
import * as llamaIndex from "npm:llamaindex@0.1.8"

In [9]:
let customLLM = new llamaIndex.OpenAI()
let customEmbedding = new llamaIndex.OpenAIEmbedding()


 Let's put the LLM and the embedding model into a `ServiceContext` object:

In [10]:
let customServiceContext = new llamaIndex.serviceContextFromDefaults({
    llm: customLLM,
    embedModel: customEmbedding
})

### Let's make our own prompt

In [11]:
let customQaPrompt = function({context = "", query = ""}) {
    return `Context information is below.
        ---------------------
        ${context}
        ---------------------
        Given the context information, answer the query.
        Include a random fact about whales in your answer.\
        The whale fact can come from your training data.
        Query: ${query}
        Answer:`
}

You need a `ResponseBuilder` that uses our prompt and our service context.

In [12]:
let customResponseBuilder = new llamaIndex.SimpleResponseBuilder(
    customServiceContext,
    customQaPrompt
)

The `responseBuilder` goes to a `synthesizer`, which also needs a service context.

In [13]:
let customSynthesizer = new llamaIndex.ResponseSynthesizer({
    responseBuilder: customResponseBuilder,
    serviceContext: customServiceContext
})

You also need a `retriever`.

In [14]:
let customRetriever = new llamaIndex.VectorIndexRetriever({
    index
})

The `synthesizer` and the `retriever` go to our query engine:

In [15]:
let customQueryEngine = new llamaIndex.RetrieverQueryEngine(
    customRetriever,
    customSynthesizer
)

### Let's check the response!

In [16]:
let response2 = await customQueryEngine.query({
    query: "What does the author think of college?"
})

In [17]:
console.log(response2.toString())

The author has a mixed opinion of college. They acknowledge that it was the right decision for them to drop out and that they were not particularly motivated by the curriculum. They also express resentment and suspicion towards higher education due to negative experiences. However, they do recognize the privilege of having access to college and acknowledge that for some people, getting a degree is necessary to gain access to assumed competence in the industry.

Random whale fact: Whales are known to communicate with each other using complex vocalizations, with some species even having distinct dialects within their populations.
